In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [2]:
def decomp1d(n, num_blocks, sub_mat_index):
    """_summary_

    Args:
        n (int): matrix lenght
        num_blocks (int): number of blocks to split into
        sub_mat_index (int): index of sub matrix row

    Returns:
        _type_: _description_
    """
    remainder = n % num_blocks
    base = n // num_blocks

    if (sub_mat_index < remainder):
        s = sub_mat_index * (base+1)
        e = s + base+1
        return s, e  # start, end
    else:
        s = (remainder * (base+1)) + (max(sub_mat_index-remainder, 0) * base)
        e = s + base
        return s, e 
   
def decomp_matrix(A, block_rows, block_cols):
    block_A = [[0]*block_cols for i in range(block_rows)]
    #print(a)
    #m, n = A.shape
    m, n = len(A), len(A[0])
    
    for i in range(block_rows):
        s1, e1 = decomp1d(m, block_rows, i)
        for j in range(block_cols):
            s2, e2 = decomp1d(n, block_cols, j)
            sub_block_rows = e1 - s1
            sub_block_cols = e2 - s2
            block_A[i][j] = np.zeros((sub_block_rows, sub_block_cols))
            for a in range(sub_block_rows):
                for b in range(sub_block_cols):
                    block_A[i][j][a][b] = A[s1 + a][s2 + b]

    #print(a)
    return block_A

def comp_matrix(m, n, block_mat):
    B = np.zeros((m, n))
    block_rows = len(block_mat)
    block_cols = len(block_mat[0])
    
    #print(block_rows, block_cols)
    
    row_step = 0
    col_step = 0
    sub_rows = 0
    sub_cols = 0
    
    for i in range(block_rows):
        for j in range(block_cols):
            block = np.array(block_mat[i][j])
            #print(f"block.shape{block.shape}")
            #print(f"block{block}")
            
            sub_rows, sub_cols = len(block), len(block[0])
            #print(f"sub_rows, sub_cols = {sub_rows, sub_cols}")
            #print(f"row_step, col_step ={row_step, col_step}")
            for a in range(sub_rows):
                for b in range(sub_cols):
                    B[row_step + a][col_step + b] = block[a][b]
                    
            col_step += sub_cols
        row_step += sub_rows
        col_step = 0
            
    return B

def mgs(A):
    """Calculates QR decomposition of a matrix A using the modified Gram Schmidt algorithm.

    Args:
        A (numpy array): numpy matrix

    Returns:
        tuple: the pair Q, R
    """
    #m, n = A.shape
    m, n = len(A), len(A[0])
    print(f"m, n = {m}, {n}")
    
    a = [A[:, i] for i in range(n)]
    
    Q = np.zeros((m, n))
    q = [Q[:, i] for i in range(n)]
    
    V = np.zeros((m, n))
    v = [V[:, i] for i in range(n)]
    
    R = np.zeros((n, n))
    r = [R[:, i] for i in range(n)]
    
    for i in range(n):
        v[i] = a[i]
    for i in range(n):
        r[i][i] = np.linalg.norm(v[i])
        q[i] = v[i] / r[i][i]
        for j in range(i, n):
            r[i][j] = q[i].T @ v[j]
            v[j] = v[j] - (r[i][j]*q[i])
    
    Q = np.reshape(np.array(q), (m,n))
    R = np.reshape(np.array(r), (n,n))
    
    return Q, R

In [21]:
def TSQR(A, max_block_rows):
    #m, n = A.shape
    m, n = len(A), len(A[0])
    print(f"m, n  = {m, n}")
    Q = np.eye(m)
    R = decomp_matrix(A, max_block_rows, 1)
    
    rows_num = max_block_rows
    passed = 0
    while rows_num > 1:
        passed += 1
        print(f"R shape={len(R), len(R[0])}")
        
        Q_temp = [[None]*rows_num for _ in range(rows_num)]
        R_temp = [None]*rows_num
        
        #print(f"Q_temp shape ={len(Q_temp), len(Q_temp[0])}")
        #print(f"len(R_temp)={len(R_temp)}")
        
        for i in range(rows_num):
            Qi, Ri = np.linalg.qr(R[i][0] , mode="complete")
            #print(f"R[i][0]={R[i][0]}")
            #Qi, Ri = mgs(R[i][0])
            
            R_temp[i] = Ri
            Q_temp[i][i] = Qi
            
            print(f"i={i}")
            print(f"Qi.shape={Qi.shape}")
            print(f"Ri.shape={Ri.shape}")
            
        for i in range(rows_num):
            for j in range(rows_num):
                if i!=j:
                    #m0, n0 = Q_temp[i][i].shape
                    m0, n0 = len(Q_temp[i][i]), len(Q_temp[i][i][0])
                    Q_temp[i][j] = np.zeros((m0, n0))
           
        
        #for i in range(rows_num):
        #    for j in range(rows_num):
        #       print(f"Q[{i}][{j}]={Q_temp[i][j]}")
        
        #print(f"Q_temp={Q_temp[i][j]}")
        
        #Q = Q @ comp_matrix(m, n*rows_num, Q_temp)
        
        #print(f"Q.shape, comp_matrix.shape = {Q.shape, comp_matrix(m, m, Q_temp).shape}")
        Q = Q @ comp_matrix(m, m, Q_temp)
            
        #print(f"R_temp={R_temp[0]}")
        
        rows_num = rows_num // 2
        R = [None]*rows_num
        print(f"R={R}")
        for i in range(rows_num):
            R1 = R_temp[i*2]
            R2 = R_temp[i*2 + 1]
            print(f"len(R1), len(R2) = {len(R1), len(R2)}")
            m0 = len(R1) + len(R2)
            n0 = n
            print(f"m0, n0 = {m0, n0}", f"i={i}")
            
            R[i] = [comp_matrix(m0, n0, [[R1], [R2]])]
        
        print(f"pass = {passed}")
        print("")
        
    Q_temp, R_temp = np.linalg.qr(R[0][0])
    Q = Q @ Q_temp
    R = R_temp
    
    return Q, R

In [76]:
A = np.random.rand(40, 4)

npQ, npR = np.linalg.qr(A)
Q, R = TSQR(A, 4)

print(f"Q.shape={Q.shape}")
print(f"R.shape={R.shape}")

print(f"MSE(Q - npQ) = {np.mean(np.square(Q - npQ))}")
print(f"MSE(R - npR) = {np.mean(np.square(R - npR))}")

m, n  = (40, 4)
R shape=(4, 1)
i=0
Qi.shape=(10, 10)
Ri.shape=(10, 4)
i=1
Qi.shape=(10, 10)
Ri.shape=(10, 4)
i=2
Qi.shape=(10, 10)
Ri.shape=(10, 4)
i=3
Qi.shape=(10, 10)
Ri.shape=(10, 4)
R=[None, None]
len(R1), len(R2) = (10, 10)
m0, n0 = (20, 4) i=0
len(R1), len(R2) = (10, 10)
m0, n0 = (20, 4) i=1
pass = 1

R shape=(2, 1)
i=0
Qi.shape=(20, 20)
Ri.shape=(20, 4)
i=1
Qi.shape=(20, 20)
Ri.shape=(20, 4)
R=[None]
len(R1), len(R2) = (20, 20)
m0, n0 = (40, 4) i=0
pass = 2

Q.shape=(40, 4)
R.shape=(4, 4)
MSE(Q - npQ) = 0.024999999999999994
MSE(R - npR) = 1.6880821390440681


In [239]:
num = 6
#A = np.eye(num)
A = np.random.rand(num*2, num)
num_cols = 4
num_rows = 3

A_decomp = decomp_matrix(A, block_rows=num_rows, block_cols=num_cols)
for i in range(num_rows):
    for j in range(num_cols):
        print(i, j)
        print(A_decomp[i][j])
"""
"""
        
A_recomp = comp_matrix(num, num, A_decomp)
print(A_recomp)

0 0
[[8.05820327e-01 3.16420216e-04]
 [2.05154148e-01 4.00054372e-01]
 [2.69119587e-01 2.51093936e-01]
 [9.37040347e-01 4.21486635e-02]]
0 1
[[0.10069171 0.51120311]
 [0.54313252 0.73103081]
 [0.45930409 0.97691732]
 [0.15872487 0.67092943]]
0 2
[[0.25327994]
 [0.70366308]
 [0.05937462]
 [0.12265918]]
0 3
[[0.08940432]
 [0.1867708 ]
 [0.98229866]
 [0.52417089]]
1 0
[[0.59721053 0.21556088]
 [0.65352341 0.49382522]
 [0.58165693 0.10375922]
 [0.42563954 0.09602589]]
1 1
[[0.85492079 0.24355692]
 [0.86688393 0.83933736]
 [0.36457476 0.9434652 ]
 [0.71684287 0.01666586]]
1 2
[[0.70527155]
 [0.43948281]
 [0.09986251]
 [0.98621812]]
1 3
[[0.9186759 ]
 [0.03534241]
 [0.5266008 ]
 [0.35238568]]
2 0
[[0.79198223 0.46441414]
 [0.30576641 0.70149342]
 [0.41577353 0.65807411]
 [0.55100927 0.4069521 ]]
2 1
[[0.14264957 0.83523697]
 [0.45526898 0.01497796]
 [0.35465273 0.04895121]
 [0.93720379 0.42677154]]
2 2
[[0.32979196]
 [0.96594594]
 [0.54328281]
 [0.78613589]]
2 3
[[0.7343909 ]
 [0.97423635]
 

IndexError: index 6 is out of bounds for axis 0 with size 6

In [113]:
my_list = [i for i in range(7)]

blocks = 4
for i in range(blocks):
    s, e = decomp1d(len(my_list), blocks, i)
    print(s, e)
    print(my_list[s:e])

0 2
[0, 1]
2 4
[2, 3]
4 6
[4, 5]
6 7
[6]
